In [3]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import numpy as np
import sys
sys.path.append( '/Users/sultandaniels/Documents/ENGN1000/mealmaker_real/src/autochef/data' )

import ingredients_list as all_ing
all_ing = all_ing.ingredients
np.savetxt("ingredients.csv", all_ing, delimiter=", ", fmt = '%s')
import actions_list
cooking_verbs = actions_list.cooking_verbs
np.savetxt("cooking_actions.csv", cooking_verbs, delimiter=", ", fmt = '%s')
heat = actions_list.heat
np.savetxt("heating_actions.csv", heat, delimiter=", ", fmt = '%s')
cool = actions_list.cool
np.savetxt("cooling_actions.csv", cool, delimiter=", ", fmt = '%s')
prepare = actions_list.prepare
np.savetxt("preparing_actions.csv", prepare, delimiter=", ", fmt = '%s')




# data

In [47]:
f = open("/Users/sultandaniels/Documents/ENGN1000/mealmaker/src/autochef/data/1M_recipes/layer1.json", "r")
data = json.load(f)

f.close()

In [48]:
#print the ingredients of each recipe
# print(data[200]["ingredients"][0]["text"])
import numpy as np



# ingredient list

In [49]:
#creates ingredient list for a recipe
#inputs recipe outputs ingredient list
def ing_list_fun(rec):
    ings = []
    ings_uncleaned = []
    for ing in rec["ingredients"]:
        ing = word_tokenize(ing["text"])
        ing = [word.lower() for word in ing] #lowercase everything
        its = ing.count("(") + ing.count(")") + ing.count(",") + ing.count("ounce")\
              + ing.count("cup")+ ing.count("teaspoon")+ ing.count("tablespoon")\
              + ing.count("quart")+ ing.count("gallon")#count number of common unnecessary phrases
        for i in range(its): #for the number of unnecessary characters, use the remove function for each
            try:
                ing.remove("(")
            except ValueError:
                try:
                    ing.remove(")")
                except ValueError:
                    try:
                        ing.remove(",")
                    except ValueError:
                        try:
                            ing.remove("ounce")
                        except ValueError:
                            try:
                                ing.remove("cup")
                            except ValueError:
                                try:
                                    ing.remove("teaspoon")
                                except ValueError:
                                    try:
                                        ing.remove("tablespoon")
                                    except ValueError:
                                        try:
                                            ing.remove("quart")
                                        except ValueError:
                                            ing.remove("gallon")
        its = ing.count("spoon") + ing.count("note") + ing.count("see") + ing.count("page") + ing.count("recipe")#count number of common unnecessary phrases
        for i in range(its): #for the number of unnecessary characters, use the remove function for each
            try:
                ing.remove("recipe")
            except ValueError:
                try:
                    ing.remove("page")
                except ValueError:
                    try:
                        ing.remove("see")
                    except ValueError:
                        try:
                            ing.remove("note")
                        except ValueError:
                            ing.remove("spoon")

        ing = pos_tag(ing)
        for word in ing:
            ings_uncleaned.append(word)
            if word[1] == "NNP" or word[1] == "NN":
                ings.append(word[0])
    return ings

ing_list_fun(data[1])


['c.',
 'elbow',
 'macaroni',
 'c.',
 'c.',
 'celery',
 'c.',
 'pepper',
 'tbsp',
 'mayonnaise',
 'dressing',
 'tbsp',
 'vinegar',
 'salt',
 'dill']

## ingredient list of all recipes

In [50]:
ings_recs = []
for rec in data:
    ings_recs.append(ing_list_fun(rec))


# Adjacency matrices

## per recipe

### actions to ingredients

In [51]:
## A-I (action to ingredient) matrix for a recipe
#inputs a recipe, its recipe_list, and action array outputs an action to ingredient adjacency matrix
def act_ing_rec(rec,rec_list,acts):
    adj = np.zeros([len(acts),len(rec_list)])
    for i in range(0, len(acts)):
        for j in range(0, len(rec_list)):
            for instr in rec["instructions"]:
                if acts[i] in instr["text"] and rec_list[j] in instr["text"]:
                    adj[i,j] = 1
    return adj

# act_ing_rec(data[0],ing_list_fun(data[0]),cooking_verbs)

### ingredients to ingredients

In [52]:
## I-I matrix for a recipe
#inputs a recipe and its ingredient list, outputs an ingredient to ingredient adjacency matrix
def ing_ing_rec(rec,rec_list):
    adj = np.zeros([len(rec_list),len(rec_list)])
    for i in range(0, len(rec_list)):
        for j in range(0, len(rec_list)):
            for instr in rec["instructions"]:
                if rec_list[i] in instr["text"] and rec_list[j] in instr["text"]:
                    adj[i,j] = 1
    return adj

## entire database

### Actions to all ingredients

In [53]:
#inputs the recipe database, list of all ingredients, and the list of the ingredient list of each recipes, and action list
#outputs an adjacency matrix that shows the frequency of ingredients having a certain action done to them
def act_ing_all(data, all_ings, rec_ings, acts):
    adj = np.zeros([len(acts), len(all_ings)])
    ing_ind = 0
    for k in range(0,len(rec_ings)): #iterate through all of the recipes, updating adj each time
        ati_mat = act_ing_rec(data[k],rec_ings[k],acts) #action to ingredients adj matrix for recipe k
        for i in range(0, len(all_ings)): #check each ingredient
            if all_ings[i] in rec_ings[k]: #if ingredient in the recipe get its ingr index
                ing_ind = rec_ings[k].index(all_ings[i])
                # add ingr column to column of entire adj
                adj[:,i] = adj[:,i] + ati_mat[:,ing_ind]

    return adj

# print(np.amax(act_ing_all(data,all_ing,ings_recs, cooking_verbs)))
# print(act_ing_rec(data[0],ing_list_fun(data[0]),cooking_verbs)[0][2])


406301.0


### ingredients to ingredients

In [55]:
#inputs the recipe database, list of all ingredients, and the list of the ingredient list of each recipes.
#outputs an adjacency matrix that shows the frequency of ingredients being mixed together
def ing_ing_all(data, all_ings, rec_ings):
    adj = np.zeros([len(all_ings), len(all_ings)])
    ing_ind = 0
    for k in range(0,len(rec_ings)): #iterate through all of the recipes, updating adj each time
        iti_mat = ing_ing_rec(data[k],rec_ings[k])
        nonz_els = np.transpose(np.nonzero(iti_mat)) #indices of non-zero entries of iti_mat
        nze_shape = nonz_els.shape
        for i in range(0, nze_shape[0]):
            ing1 = rec_ings[k][nonz_els[i,0]] #first ingredient that was mixed
            ing2 = rec_ings[k][nonz_els[i,1]] #second ingredient that was mixed
            try:
                ing1_ind = all_ings.index(ing1)
                ing2_ind = all_ings.index(ing2)
                adj[ing1_ind,ing2_ind] = adj[ing1_ind,ing2_ind] + 1
            except ValueError:
                # print("dirty ingredient")
                continue
    return adj

# print(np.max(ing_ing_all(data,all_ing,ings_recs)))
# print(np.min(ing_ing_all(data,all_ing,ings_recs)))
# print(ing_ing_all(data,all_ing,ings_recs).shape)
# print(np.nonzero(ing_ing_all(data,all_ing,ings_recs) - np.transpose(ing_ing_all(data,all_ing,ings_recs)))) #truly a symmetric matrix

### Action group to ingredients

In [56]:
#inputs the action to ingredient adjacency matrix, action list, and action group list of lists
#outputs an adjacency matrix that shows the frequency of ingredients having a certain action group done to them
def actgrp_ing_all(act_ing, acts, actgrps):
    mat_sh = act_ing.shape
    adj = np.zeros([3, mat_sh[1]])
    i = 0
    while i < 3:
        for act in actgrps[i]: # for action in specific subgroup
            act_ind = acts.index(act) # get overall index of action
            adj[i,:] = adj[i,:] + act_ing[act_ind,:] # add entire row in the A*I matrix
            # for that action to the subgroup's row
        i = i + 1
    return adj

# print(np.amax(actgrp_ing_all(act_ing_all(data, all_ing, ings_recs, cooking_verbs), cooking_verbs, [heat, cool, prepare])))

# Write Adj Matrices into files

In [57]:
#act to ing
act_ing = act_ing_all(data, all_ing, ings_recs, cooking_verbs)
np.savetxt("actions_to_ingredients_matrix.csv", act_ing, delimiter=", ", fmt = '%f')
act_ing_thresh = act_ing
act_ing_thresh[act_ing_thresh < 2] = 0
np.savetxt("actions_to_ingredients_thresholded_matrix.csv", act_ing_thresh, delimiter=", ", fmt = '%f')

#ing to ing
ing_ing = ing_ing_all(data, all_ing, ings_recs)
np.savetxt("ingredients_to_ingredients_matrix.csv", ing_ing, delimiter=", ", fmt = '%f')
ing_ing_thresh = ing_ing
ing_ing_thresh[ing_ing_thresh < 2] = 0
np.savetxt("ingredients_to_ingredients_thresholded_matrix.csv", ing_ing_thresh, delimiter=", ", fmt = '%f')

#actgrp to ing
actgrp_ing = actgrp_ing_all(act_ing, cooking_verbs, [heat, cool, prepare])
np.savetxt("action_groups_to_ingredients_heat_cool_prepare_matrix.csv", actgrp_ing, delimiter=", ", fmt = '%f')
actgrp_ing_thresh = actgrp_ing
actgrp_ing_thresh[actgrp_ing_thresh < 2] = 0
np.savetxt("action_groups_to_ingredients_heat_cool_prepare_thresholded_matrix.csv", actgrp_ing_thresh, delimiter=", ", fmt = '%f')



In [58]:
print(ing_ing.shape)

(6714, 6714)


In [60]:
newing = np.loadtxt("ingredients_to_ingredients_matrix.csv", delimiter=",", dtype=float)

In [61]:
print(newing.shape)

(6714, 6714)


## action group indices

In [1]:
# inputs cooking list and prepare action list and returns list of indices of preparing actions
def prep_inds(cooking, preps):
    prep_inds = []
    for prep in preps:
        prep_inds.append(cooking.index(prep))

    return prep_inds


# inputs cooking list and heating action list and returns list of indices of heating actions
def heat_inds(cooking, heats):
    heat_inds = []
    for heat in heats:
        heat_inds.append(cooking.index(heat))

    return heat_inds

In [4]:
prep_inds = prep_inds(cooking_verbs,prepare)
np.savetxt("prepare_actions_indices.csv", prep_inds, delimiter=", ", fmt = '%f')

heat_inds = heat_inds(cooking_verbs, heat)
np.savetxt("heat_actions_indices.csv", heat_inds, delimiter=", ", fmt = '%f')